## import thư viện

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageSequence
from tqdm import tqdm

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models import vit_b_16, ViT_B_16_Weights

In [ ]:
# import sys
# # Uninstall the potentially incorrect 'clip' package
# #!{sys.executable} -m pip uninstall -y clip
# # Install the correct OpenAI CLIP package
# !{sys.executable} -m pip install openai-clip

# # Force Python to reload the clip module by removing it from sys.modules
# if 'clip' in sys.modules:
#     del sys.modules['clip']

# # Re-import clip to ensure the newly installed module is used
# import clip

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
# class BannerContrastDataset(Dataset):
#     def __init__(self, image_dir, label_file, transform=None):
#         self.image_dir = image_dir
#         self.labels_df = pd.read_csv(label_file)
#         self.transform = transform
#         self.extensions = ['jpg', 'jpeg', 'png', 'gif']

#     def find_image_file(self, index):
#         base = os.path.join(self.image_dir, str(index))
#         for ext in self.extensions:
#             filepath = f"{base}.{ext}"
#             if os.path.exists(filepath):
#                 return filepath
#         raise FileNotFoundError(f"Không tìm thấy ảnh cho index: {index}")

#     def load_image(self, path):
#         img = Image.open(path)

#         # xử lý GIF động
#         if getattr(img, "is_animated", False):
#             img = ImageSequence.Iterator(img).__next__()

#         return img.convert("RGB")

#     def __len__(self):
#         return len(self.labels_df)

#     def __getitem__(self, idx):
#         row = self.labels_df.iloc[idx]

#         img_path = self.find_image_file(row['index'])
#         image = self.load_image(img_path)
#         label = float(row['label'])

#         if self.transform:
#             image = self.transform(image)

#         return image, label


## data reading

In [ ]:
def get_image_path(folder, index):
    # chỉ cho phép 3 định dạng này
    valid_exts = ["jpeg", "jpg", "png", "gif"]

    for ext in valid_exts:
        path = os.path.join(folder, f"{index}.{ext}")
        if os.path.exists(path):
            return path
    return None

In [ ]:
# hàm tạo danh sách path ảnh và label
def load_dataset(img_folder, label_file):
  df = pd.read_csv(label_file)

  img_paths = []
  labels = []

  for _, row in df.iterrows():
    idx = int(row['index'])
    label = float(row['label'])

    img_path = get_image_path(img_folder, idx)

    if img_path is None:
      print(f'Lỗi: Không tìm thấy ảnh index = {idx}')
      continue

    img_paths.append(img_path)
    labels.append(label) # ép nhãn về kiểu float phục vụ cho regression assess

  return img_paths, labels

## bannerDataset init

In [ ]:
class BannerDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels  # float 1-5
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        # label = self.labels[idx]
        # label = torch.tensor(self.labels[idx], dtype=torch.float)
        label = torch.tensor([float(self.labels[idx])], dtype=torch.float)
        img = Image.open(img_path)
        if getattr(img, "is_animated", False):
            img = ImageSequence.Iterator(img).__next__()

        #img = img.convert("RGB")
        img = img.convert("RGBA").convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
# class BannerContrastDatasetFromDF(Dataset):
#     def __init__(self, image_dir, df, transform=None):
#         self.image_dir = image_dir
#         self.df = df.reset_index(drop=True)
#         self.transform = transform
#         self.extensions = ['jpg','jpeg','png','gif']

#     def find_image_file(self, index):
#         base = os.path.join(self.image_dir, str(index))
#         for ext in self.extensions:
#             path = f"{base}.{ext}"
#             if os.path.exists(path):
#                 return path
#         raise FileNotFoundError(f"Không tìm thấy ảnh cho index {index}")

#     def load_image(self, path):
#         img = Image.open(path)
#         if getattr(img, "is_animated", False):
#             img = ImageSequence.Iterator(img).__next__()
#         return img.convert("RGB")

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img_path = self.find_image_file(row['index'])
#         image = self.load_image(img_path)
#         label = torch.tensor([float(row['label'])], dtype=torch.float32)
#         if self.transform:
#             image = self.transform(image)
#         return image, label


In [ ]:
# class RegressionHead(nn.Module):
#     def __init__(self, in_dim):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(in_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(256, 1)
#         )

#     def forward(self, x):
#         return self.net(x)

## Regression head

In [ ]:
class RegressionHead(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),

            # nn.Linear(256, 64),
            # nn.ReLU(),
            # nn.Dropout(0.1),


            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# def get_efficientnet_model():
#     weights = EfficientNet_B0_Weights.IMAGENET1K_V1
#     effnet = efficientnet_b0(weights=weights)

#     for p in effnet.parameters():
#         p.requires_grad = False

#     effnet.classifier = RegressionHead(
#         in_dim=effnet.classifier[1].in_features
#     )
#     return effnet.to(device)

## Model

In [ ]:
def get_efficientnet_model():
    # weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    # effnet = efficientnet_b0(weights=weights)

    # for p in effnet.parameters():
    #     p.requires_grad = False

    # effnet.classifier = RegressionHead(
    #     in_dim=effnet.classifier[1].in_features
    # )

#----------------------------------------------------
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    effnet = efficientnet_b0(weights=weights)

    # ------------------------
    # Freeze toàn bộ trước
    # ------------------------
    for p in effnet.parameters():
        p.requires_grad = False

    # ------------------------
    # Unfreeze một vài block cuối
    # EfficientNet-B0 features có dạng:
    # effnet.features[0] -> effnet.features[8]
    # ------------------------
    UNFREEZE_FROM = 5      # mở từ block 6 → 8 (3 block cuối)
    for idx in range(UNFREEZE_FROM, len(effnet.features)):
        for p in effnet.features[idx].parameters():
            p.requires_grad = True

    # ------------------------
    # Replace classifier
    # ------------------------
    in_dim = effnet.classifier[1].in_features   # 1280
    effnet.classifier = RegressionHead(in_dim)

    # Head mới cần phải learn
    for p in effnet.classifier.parameters():
        p.requires_grad = True

    return effnet.to(device)

In [ ]:
# def get_vit_model():
#     weights = ViT_B_16_Weights.IMAGENET1K_V1
#     vit = vit_b_16(weights=weights)

#     for p in vit.parameters():
#         p.requires_grad = False

#     vit.heads = RegressionHead(
#         in_dim=vit.heads.head.in_features
#     )
#     return vit.to(device)


In [ ]:
def get_vit_model():
    # weights = ViT_B_16_Weights.IMAGENET1K_V1
    # vit = vit_b_16(weights=weights)

    # for p in vit.parameters():
    #     p.requires_grad = False

    # vit.heads = RegressionHead(
    #     in_dim=vit.heads.head.in_features
    # )

    #--------------------------------

    weights = ViT_B_16_Weights.IMAGENET1K_V1
    vit = vit_b_16(weights=weights)

    # 1. Freeze toàn bộ backbone trước
    for p in vit.parameters():
        p.requires_grad = False

    # 2. Unfreeze một vài layer cuối của Transformer Encoder
    # ViT-B/16 có 12 layer encoder: layers[0..11]
    UNFREEZE_FROM = 5   # mở 3 layer cuối (layers 9, 10, 11)
    for idx in range(UNFREEZE_FROM, len(vit.encoder.layers)):
        for p in vit.encoder.layers[idx].parameters():
            p.requires_grad = True

    # 3. Replace head bằng regression head
    in_dim = vit.heads.head.in_features   # thường là 768
    vit.heads = RegressionHead(in_dim)

    # 4. Cho head mới học
    for p in vit.heads.parameters():
        p.requires_grad = True

    return vit.to(device)

In [ ]:
def r2_score(preds, targets):
    """
    preds, targets: Tensor shape (batch, 1)
    """
    ss_res = torch.sum((targets - preds) ** 2)
    ss_tot = torch.sum((targets - torch.mean(targets)) ** 2)
    return 1 - ss_res / ss_tot

## train funct

In [ ]:
# # def train_one_epoch(model, loader, optimizer, loss_fn):
# #     model.train()
# #     total_loss = 0
# #     total_r2 = 0

# #     for imgs, labels in loader:
# #         imgs, labels = imgs.to(device), labels.to(device)

# #         optimizer.zero_grad()
# #         preds = model(imgs)
# #         loss = loss_fn(preds, labels)
# #         loss.backward()
# #         optimizer.step()

# #         total_loss += loss.item()
# #         total_r2 += r2_score(preds, labels).item()

# #     return total_loss / len(loader), total_r2 / len(loader)

# def train_one_epoch(model, loader, optimizer, loss_fn):
#     model.train()
#     total_loss = 0
#     total_r2 = 0

#     pbar = tqdm(loader, desc="Train", leave=False)
#     for imgs, labels in loader:
#         imgs, labels = imgs.to(device), labels.to(device)

#         optimizer.zero_grad()
#         preds = model(imgs)
#         loss = loss_fn(preds, labels)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()
#         #total_r2 += r2_score(preds, labels).item()
#         r2 = r2_score(preds, labels)
#         total_r2 += r2.item()
#         pbar.set_postfix({"Loss": f"{loss.item():.4f}", "R2": f"{r2.item():.4f}"})

#     return total_loss / len(loader), total_r2 / len(loader)


In [ ]:
# không in dùng cái này
def train_one_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    total_mae = 0
    total_r2 = 0

    mae_fn = nn.L1Loss()

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_mae += mae_fn(preds, labels).item()
        total_r2 += r2_score(preds, labels).item()

    n = len(loader)
    return total_loss / n, total_mae / n, total_r2 / n

In [ ]:
# # def evaluate(model, loader, loss_fn):
# #     model.eval()
# #     total_loss = 0
# #     total_r2 = 0

# #     with torch.no_grad():
# #         for imgs, labels in loader:
# #             imgs, labels = imgs.to(device), labels.to(device)
# #             preds = model(imgs)
# #             loss = loss_fn(preds, labels)

# #             total_loss += loss.item()
# #             total_r2 += r2_score(preds, labels).item()

# #     return total_loss / len(loader), total_r2 / len(loader)

# def evaluate(model, loader, loss_fn):
#     model.eval()
#     total_loss = 0
#     total_r2 = 0

#     pbar = tqdm(loader, desc="Val  ", leave=False)
#     with torch.no_grad():
#         for imgs, labels in pbar:
#             imgs, labels = imgs.to(device), labels.to(device)
#             preds = model(imgs)
#             loss = loss_fn(preds, labels)

#             total_loss += loss.item()
#             #total_r2 += r2_score(preds, labels).item()
#             r2 = r2_score(preds, labels)
#             total_r2 += r2.item()
#             pbar.set_postfix({"Loss": f"{loss.item():.4f}", "R2": f"{r2.item():.4f}"})

#     return total_loss / len(loader), total_r2 / len(loader)


## evaluate funct

In [ ]:
# không in dùng cái này
def evaluate(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    total_mae = 0
    total_r2 = 0

    mae_fn = nn.L1Loss()

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs)
            loss = loss_fn(preds, labels)

            total_loss += loss.item()
            total_mae += mae_fn(preds, labels).item()
            total_r2 += r2_score(preds, labels).item()

    n = len(loader)
    return total_loss / n, total_mae / n, total_r2 / n

## Early stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# def run_training(df_train, df_val, image_dir, model_name="effnet",
#                  batch_size=16, epochs=10, lr=1e-3, early_stopping_patience=3):

#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225]),
#     ])

#     train_ds = BannerContrastDatasetFromDF(image_dir, df_train, transform)
#     val_ds   = BannerContrastDatasetFromDF(image_dir, df_val, transform)

#     train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
#     val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#     # ---- Model ----
#     if model_name == "effnet":
#         model = get_efficientnet_model()
#         params = model.classifier.parameters()
#     elif model_name == "vit":
#         model = get_vit_model()
#         params = model.heads.parameters()
#     else:
#         raise ValueError("model_name phải là effnet hoặc vit")

#     # ---- Optimizer, Loss ----
#     loss_fn = nn.MSELoss()
#     optimizer = torch.optim.Adam(params, lr=lr)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
#     early_stopping = EarlyStopping(patience=early_stopping_patience)

#     best_val_loss = float('inf')  # dùng để lưu model tốt nhất

#     # ---- Train loop ----
#     for epoch in range(1, epochs + 1):
#         train_loss, train_r2 = train_one_epoch(model, train_loader, optimizer, loss_fn)
#         val_loss, val_r2 = evaluate(model, val_loader, loss_fn)

#         scheduler.step()

#         #print(f"Epoch {epoch}/{epochs} - Train: {train_loss:.4f} | Val: {val_loss:.4f}")
#         print(f"Epoch {epoch}/{epochs} | "
#               f"Train Loss: {train_loss:.4f} | Train R2: {train_r2:.4f} || "
#               f"Val Loss: {val_loss:.4f} | Val R2: {val_r2:.4f}")

#         # ---- Lưu model khi val_loss cải thiện ----
#         #torch.save(model.state_dict(), f"/content/drive/MyDrive/UIT/DS201/final_project/model/{model_name}_best.pth")
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), f"/content/drive/MyDrive/UIT/DS201/final_project/model/{model_name}_best.pth")
#             #print(f"  --> Model improved. Saved best model at epoch {epoch}")

#         # ---- Kiểm tra early stopping ----
#         early_stopping(val_loss)
#         if early_stopping.early_stop:
#           print(f"Early stopping at epoch {epoch}")
#           break

#     print("Training completed!")


## run train funct

In [ ]:
def run_training(train_loader, val_loader, model_name="effnet",
                 epochs=10, lr=1e-3, early_stopping_patience=3):
    # ---- TEST DATALOADER ----
    print("Testing DataLoader batch load...")
    for i, (imgs, labels) in enumerate(train_loader):
        print("Batch", i, "loaded!")
        break
    print("Done testing. Bắt đầu train...\n")

    # ---- Model ----
    # if model_name == "effnet":
    #     model = get_efficientnet_model()
    #     params = model.classifier.parameters()
    # elif model_name == "vit":
    #     model = get_vit_model()
    #     params = model.heads.parameters()
    # else:
    #     raise ValueError("model_name phải là effnet hoặc vit")

    if model_name == "effnet":
        model = get_efficientnet_model()
    elif model_name == "vit":
        model = get_vit_model()
    else:
        raise ValueError("model_name phải là effnet hoặc vit")

    # Lấy toàn bộ parameters có requires_grad = True
    params = filter(lambda p: p.requires_grad, model.parameters())

    # ---- Optimizer, Loss ----
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(params, lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    early_stopping = EarlyStopping(patience=early_stopping_patience)

    best_val_loss = float('inf')  # dùng để lưu model tốt nhất

    # ---- Train loop ----
    for epoch in range(1, epochs + 1):
        # train_loss, train_r2 = train_one_epoch(model, train_loader, optimizer, loss_fn)
        # val_loss, val_r2 = evaluate(model, val_loader, loss_fn)

        train_loss, train_mae, train_r2 = train_one_epoch(model, train_loader, optimizer, loss_fn)
        val_loss, val_mae, val_r2 = evaluate(model, val_loader, loss_fn)

        scheduler.step()

        #print(f"Epoch {epoch}/{epochs} - Train: {train_loss:.4f} | Val: {val_loss:.4f}")
        print(f"Epoch {epoch}/{epochs} | "
              f"Train Loss(MSE): {train_loss:.4f} | Train MAE: {train_mae:.4f} | Train R2: {train_r2:.4f} || "
              f"Val Loss(MSE): {val_loss:.4f} | Val MAE: {val_mae:.4f} | Val R2: {val_r2:.4f}")

        # ---- Lưu model khi val_loss cải thiện ----
        #torch.save(model.state_dict(), f"/content/drive/MyDrive/UIT/DS201/final_project/model/{model_name}_best.pth")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"/content/drive/MyDrive/UIT/DS201/final_project/model/{model_name}_6.pth")
            #print(f"  --> Model improved. Saved best model at epoch {epoch}")

        # ---- Kiểm tra early stopping ----
        early_stopping(val_loss)
        if early_stopping.early_stop:
          print(f"Early stopping at epoch {epoch}")
          break

    print("Training completed!")


## transform

In [ ]:
# ------ xử lý dữ liệu trước khi đưa vào model --------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

## main

### data loader

In [ ]:
# load dataset
label_file = "/content/drive/MyDrive/UIT/DS201/final_project/contrast_label_float_processed.csv"   # file Excel duy nhất
img_folder  = "/content/drive/MyDrive/Project_3/DS201/data/extracted_images"        # folder duy nhất chứa ảnh

all_paths, all_labels = load_dataset(img_folder, label_file)

In [ ]:
print(type(all_labels))

<class 'list'>


In [ ]:
# chia train, test
train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_paths, all_labels, test_size=0.1, random_state=42
)

In [ ]:
train_ds = BannerDataset(train_paths, train_labels, transform)
val_ds   = BannerDataset(test_paths, test_labels, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True, persistent_workers = True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False,
                        num_workers=4, pin_memory=False, persistent_workers = True)

### train

effnet1:  
freeze from 6  
512 (0.1) - 128 (0.2) - 1  

Epoch 11/30 | Train Loss(MSE): 0.1617 | Train MAE: 0.3170 | Train R2: 0.6408 || Val Loss(MSE): 0.3620 | Val MAE: 0.4823 | Val R2: 0.1950  

12min

In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=30, early_stopping_patience=3)

vit_1:  
freeze from 9  
512 (0.1) - 128 (0.2) - 1  

Epoch 14/30 | Train Loss(MSE): 0.1370 | Train MAE: 0.2935 | Train R2: 0.6976 || Val Loss(MSE): 0.3766 | Val MAE: 0.4839 | Val R2: 0.1382  


19min

In [ ]:
run_training(train_loader, val_loader, model_name="vit", epochs=30, early_stopping_patience=3)

effnet2:  
freeze from 4  
512 (0.1) - 256 (0.1) - 128 (0.1) - 1  

Epoch 28/30 | Train Loss(MSE): 0.0889 | Train MAE: 0.2351 | Train R2: 0.8017 || Val Loss(MSE): 0.3341 | Val MAE: 0.4576 | Val R2: 0.2450  


15min

In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=30, early_stopping_patience=10)

effnet4:  
freeze from 6  
512 (0.1) - 256 (0.1) - 64 (0.1) - 1  

Epoch 27/30 | Train Loss(MSE): 0.1113 | Train MAE: 0.2635 | Train R2: 0.7547 || Val Loss(MSE): 0.3156 | Val MAE: 0.4475 | Val R2: 0.3120  

20min

In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=30, early_stopping_patience=10)

Testing DataLoader batch load...
Batch 0 loaded!
Done testing. Bắt đầu train...

Epoch 1/30 | Train Loss(MSE): 1.6363 | Train MAE: 0.9622 | Train R2: -2.6584 || Val Loss(MSE): 0.9135 | Val MAE: 0.7746 | Val R2: -1.1288
Epoch 2/30 | Train Loss(MSE): 0.5711 | Train MAE: 0.6000 | Train R2: -0.2440 || Val Loss(MSE): 1.2284 | Val MAE: 0.9282 | Val R2: -1.8644
Epoch 3/30 | Train Loss(MSE): 0.4487 | Train MAE: 0.5343 | Train R2: 0.0118 || Val Loss(MSE): 0.5627 | Val MAE: 0.6105 | Val R2: -0.2377
Epoch 4/30 | Train Loss(MSE): 0.3996 | Train MAE: 0.5021 | Train R2: 0.1136 || Val Loss(MSE): 0.6082 | Val MAE: 0.6389 | Val R2: -0.3570
Epoch 5/30 | Train Loss(MSE): 0.3526 | Train MAE: 0.4728 | Train R2: 0.2253 || Val Loss(MSE): 0.4614 | Val MAE: 0.5435 | Val R2: -0.0347
Epoch 6/30 | Train Loss(MSE): 0.2811 | Train MAE: 0.4159 | Train R2: 0.3690 || Val Loss(MSE): 0.4973 | Val MAE: 0.5726 | Val R2: -0.1261
Epoch 7/30 | Train Loss(MSE): 0.2434 | Train MAE: 0.3923 | Train R2: 0.4612 || Val Loss(MSE): 0

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2eea966fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2eea966fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

Epoch 15/30 | Train Loss(MSE): 0.1563 | Train MAE: 0.3113 | Train R2: 0.6461 || Val Loss(MSE): 0.3544 | Val MAE: 0.4806 | Val R2: 0.2145
Epoch 16/30 | Train Loss(MSE): 0.1512 | Train MAE: 0.3075 | Train R2: 0.6641 || Val Loss(MSE): 0.3425 | Val MAE: 0.4696 | Val R2: 0.2432
Epoch 17/30 | Train Loss(MSE): 0.1378 | Train MAE: 0.2920 | Train R2: 0.6923 || Val Loss(MSE): 0.3455 | Val MAE: 0.4706 | Val R2: 0.2319


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2eea966fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c2eea966fc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

Epoch 18/30 | Train Loss(MSE): 0.1323 | Train MAE: 0.2868 | Train R2: 0.7065 || Val Loss(MSE): 0.3332 | Val MAE: 0.4609 | Val R2: 0.2642
Epoch 19/30 | Train Loss(MSE): 0.1482 | Train MAE: 0.3025 | Train R2: 0.6715 || Val Loss(MSE): 0.3287 | Val MAE: 0.4600 | Val R2: 0.2756
Epoch 20/30 | Train Loss(MSE): 0.1382 | Train MAE: 0.2914 | Train R2: 0.6930 || Val Loss(MSE): 0.3451 | Val MAE: 0.4694 | Val R2: 0.2279
Epoch 21/30 | Train Loss(MSE): 0.1374 | Train MAE: 0.2913 | Train R2: 0.6914 || Val Loss(MSE): 0.3267 | Val MAE: 0.4536 | Val R2: 0.2834
Epoch 22/30 | Train Loss(MSE): 0.1314 | Train MAE: 0.2846 | Train R2: 0.7044 || Val Loss(MSE): 0.3412 | Val MAE: 0.4674 | Val R2: 0.2511
Epoch 23/30 | Train Loss(MSE): 0.1236 | Train MAE: 0.2768 | Train R2: 0.7236 || Val Loss(MSE): 0.3386 | Val MAE: 0.4643 | Val R2: 0.2593
Epoch 24/30 | Train Loss(MSE): 0.1199 | Train MAE: 0.2724 | Train R2: 0.7320 || Val Loss(MSE): 0.3283 | Val MAE: 0.4549 | Val R2: 0.2835
Epoch 25/30 | Train Loss(MSE): 0.1280 | T

effnet5:  
freeze from 6  
512 (0.1) - 256 (0.1) - 64 (0.1) - 1  
transform_bicubic

In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=30, early_stopping_patience=10)

Testing DataLoader batch load...
Batch 0 loaded!
Done testing. Bắt đầu train...

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 157MB/s]


Epoch 1/30 | Train Loss(MSE): 1.6343 | Train MAE: 0.9584 | Train R2: -2.6380 || Val Loss(MSE): 0.7795 | Val MAE: 0.7034 | Val R2: -0.8291
Epoch 2/30 | Train Loss(MSE): 0.6019 | Train MAE: 0.6135 | Train R2: -0.3207 || Val Loss(MSE): 0.9422 | Val MAE: 0.7731 | Val R2: -1.1479
Epoch 3/30 | Train Loss(MSE): 0.4743 | Train MAE: 0.5501 | Train R2: -0.0386 || Val Loss(MSE): 0.7420 | Val MAE: 0.7075 | Val R2: -0.6887
Epoch 4/30 | Train Loss(MSE): 0.4090 | Train MAE: 0.5055 | Train R2: 0.1005 || Val Loss(MSE): 0.5656 | Val MAE: 0.5956 | Val R2: -0.2702
Epoch 5/30 | Train Loss(MSE): 0.3879 | Train MAE: 0.4944 | Train R2: 0.1476 || Val Loss(MSE): 0.4657 | Val MAE: 0.5478 | Val R2: -0.0552
Epoch 6/30 | Train Loss(MSE): 0.2899 | Train MAE: 0.4290 | Train R2: 0.3505 || Val Loss(MSE): 0.5797 | Val MAE: 0.6223 | Val R2: -0.3481
Epoch 7/30 | Train Loss(MSE): 0.2272 | Train MAE: 0.3794 | Train R2: 0.4941 || Val Loss(MSE): 0.5732 | Val MAE: 0.6148 | Val R2: -0.3232
Epoch 8/30 | Train Loss(MSE): 0.2062 |

effnet6:  
freeze from 4  
512 (0.1) - 256 (0.1) - 64 (0.1) - 1  

Epoch 17/50 | Train Loss(MSE): 0.1631 | Train MAE: 0.3166 | Train R2: 0.6394 || Val Loss(MSE): 0.3204 | Val MAE: 0.4469 | Val R2: 0.2953  

13min

In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=50, early_stopping_patience=10)

Testing DataLoader batch load...
Batch 0 loaded!
Done testing. Bắt đầu train...

Epoch 1/50 | Train Loss(MSE): 1.8021 | Train MAE: 0.9924 | Train R2: -3.0030 || Val Loss(MSE): 0.8005 | Val MAE: 0.7002 | Val R2: -0.7350
Epoch 2/50 | Train Loss(MSE): 0.6223 | Train MAE: 0.6253 | Train R2: -0.3493 || Val Loss(MSE): 0.5225 | Val MAE: 0.5669 | Val R2: -0.1635
Epoch 3/50 | Train Loss(MSE): 0.4841 | Train MAE: 0.5545 | Train R2: -0.0505 || Val Loss(MSE): 0.9157 | Val MAE: 0.7857 | Val R2: -1.0848
Epoch 4/50 | Train Loss(MSE): 0.4101 | Train MAE: 0.5088 | Train R2: 0.0946 || Val Loss(MSE): 0.5008 | Val MAE: 0.5685 | Val R2: -0.1441
Epoch 5/50 | Train Loss(MSE): 0.3735 | Train MAE: 0.4897 | Train R2: 0.1853 || Val Loss(MSE): 0.4904 | Val MAE: 0.5504 | Val R2: -0.0569
Epoch 6/50 | Train Loss(MSE): 0.3143 | Train MAE: 0.4450 | Train R2: 0.3010 || Val Loss(MSE): 0.4441 | Val MAE: 0.5276 | Val R2: 0.0185
Epoch 7/50 | Train Loss(MSE): 0.2920 | Train MAE: 0.4307 | Train R2: 0.3464 || Val Loss(MSE): 0

effnet6:  
freeze from 5  
512 (0.1) - 256 (0.1) - 1  


In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=40, early_stopping_patience=10)

Testing DataLoader batch load...
Batch 0 loaded!
Done testing. Bắt đầu train...

Epoch 1/40 | Train Loss(MSE): 1.3599 | Train MAE: 0.8627 | Train R2: -1.9883 || Val Loss(MSE): 0.8493 | Val MAE: 0.7533 | Val R2: -1.0096
Epoch 2/40 | Train Loss(MSE): 0.4952 | Train MAE: 0.5606 | Train R2: -0.0787 || Val Loss(MSE): 0.5919 | Val MAE: 0.6262 | Val R2: -0.3239
Epoch 3/40 | Train Loss(MSE): 0.3598 | Train MAE: 0.4788 | Train R2: 0.2089 || Val Loss(MSE): 0.5286 | Val MAE: 0.5809 | Val R2: -0.1988
Epoch 4/40 | Train Loss(MSE): 0.2721 | Train MAE: 0.4146 | Train R2: 0.4078 || Val Loss(MSE): 0.6044 | Val MAE: 0.6279 | Val R2: -0.3950
Epoch 5/40 | Train Loss(MSE): 0.2588 | Train MAE: 0.4043 | Train R2: 0.4331 || Val Loss(MSE): 0.4240 | Val MAE: 0.5335 | Val R2: 0.0267
Epoch 6/40 | Train Loss(MSE): 0.2253 | Train MAE: 0.3766 | Train R2: 0.5027 || Val Loss(MSE): 0.4783 | Val MAE: 0.5595 | Val R2: -0.0782
Epoch 7/40 | Train Loss(MSE): 0.1656 | Train MAE: 0.3212 | Train R2: 0.6304 || Val Loss(MSE): 0.

vit_2:  
freeze from 5  
512 (0.1) - 256 (0.1) - 128 (0.1) - 1  

Epoch 7/30 | Train Loss(MSE): 0.3924 | Train MAE: 0.5044 | Train R2: 0.1416 || Val Loss(MSE): 0.3497 | Val MAE: 0.4813 | Val R2: 0.2386  


30min

In [ ]:
run_training(train_loader, val_loader, model_name="vit", epochs=30, early_stopping_patience=10)

effnet3:  
freeze from 6  
512 (0.1) - 256 (0.1) - 64 (0.1) - 1  

Epoch 19/50 | Train Loss(MSE): 0.1265 | Train MAE: 0.2792 | Train R2: 0.7189 || Val Loss(MSE): 0.3496 | Val MAE: 0.4722 | Val R2: 0.2155  

14min



In [ ]:
run_training(train_loader, val_loader, model_name="effnet", epochs=50, early_stopping_patience=10)